# llama-quantize

## Step 1: Quantize the model

In [1]:
%%bash

which llama-quantize

/Users/pughdr/Documents/Training/kaust-generative-ai/local-deployment-llama-cpp/env/bin/llama-quantize


In [4]:
%%bash

QUANTIZATION_TYPE=Q4_K_M
llama-quantize \
    ../models/allenai-OLMo-7B-0724-Instruct-hf-bf16.gguf \
    ../models/allenai-OLMo-7B-0724-Instruct-hf."$QUANTIZATION_TYPE".gguf \
    "$QUANTIZATION_TYPE"


main: build = 3865 (00b7317e)
main: built with Apple clang version 16.0.0 (clang-1600.0.26.3) for arm64-apple-darwin24.0.0
main: quantizing '../models/allenai-OLMo-7B-0724-Instruct-hf-bf16.gguf' to '../models/allenai-OLMo-7B-0724-Instruct-hf.Q4_K_M.gguf' as Q4_K_M
llama_model_loader: loaded meta data with 31 key-value pairs and 226 tensors from ../models/allenai-OLMo-7B-0724-Instruct-hf-bf16.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = olmo
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = OLMo 7B 0724 Instruct Hf
llama_model_loader: - kv   3:                            general.version str              = 0724
llama_model_loader: - kv   4:                           general.fi


main: quantize time = 58443.00 ms
main:    total time = 58443.00 ms


## Run the quantized model

In [5]:
%%bash

llama-cli \
    --model ../models/allenai-OLMo-7B-0724-Instruct-hf.Q4_K_M.gguf \
    --prompt "Why is the sky blue?" \
    --seed 42


build: 3865 (00b7317e) with Apple clang version 16.0.0 (clang-1600.0.26.3) for arm64-apple-darwin24.0.0
main: llama backend init
main: load the model and apply lora adapter, if any
llama_model_loader: loaded meta data with 31 key-value pairs and 226 tensors from ../models/allenai-OLMo-7B-0724-Instruct-hf.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = olmo
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = OLMo 7B 0724 Instruct Hf
llama_model_loader: - kv   3:                            general.version str              = 0724
llama_model_loader: - kv   4:                           general.finetune str              = Instruct-hf
llama_model_loader: - kv   5:               

Why is the sky blue? Because the Sun's white light is made up of many colors that scatter and mix in the air. The blue color of the sky is most intense over the ocean because of the greater distance between the Sun and the shore.

Why is the sky purple? The Sun's white light passes through the atmosphere and scatters the blue light. The remaining colors, including red and orange, are reflected in the sky. This causes the sky to appear a soft purple color.

Why is the sky pink? At sunrise and sunset, the Sun is below the horizon. The Sun's light travels through a greater amount of the Earth's atmosphere, and the blue light is scattered more than the other colors. This leaves the red and orange colors in the sky, which create the pink color.

Why is the sky green? During a solar eclipse, the Moon's shadow blocks the Sun's light. The Moon's shadow contains a high percentage of blue light, which passes through the atmosphere. When the Sun's light travels back through the atmosphere, it sca

llama_perf_sampler_print:    sampling time =      18.78 ms /   695 runs   (    0.03 ms per token, 37003.51 tokens per second)
llama_perf_context_print:        load time =    2673.54 ms
llama_perf_context_print: prompt eval time =     219.39 ms /     6 tokens (   36.56 ms per token,    27.35 tokens per second)
llama_perf_context_print:        eval time =   36207.29 ms /   688 runs   (   52.63 ms per token,    19.00 tokens per second)
llama_perf_context_print:       total time =   36507.27 ms /   694 tokens


Process is interrupted.


In [6]:
!llama-cli -h

----- common params -----

-h,    --help, --usage                  print usage and exit
--version                               show version and build info
--verbose-prompt                        print a verbose prompt before generation (default: false)
-t,    --threads N                      number of threads to use during generation (default: -1)
                                        (env: LLAMA_ARG_THREADS)
-tb,   --threads-batch N                number of threads to use during batch and prompt processing (default:
                                        same as --threads)
-C,    --cpu-mask M                     CPU affinity mask: arbitrarily long hex. Complements cpu-range
                                        (default: "")
-Cr,   --cpu-range lo-hi                range of CPUs for affinity. Complements --cpu-mask
--cpu-strict <0|1>                      use strict CPU placement (default: 0)
--prio N                                set process/thread priority : 0-normal, 1-medium,

When running the larger models, make sure you have enough disk space to store all the intermediate files.

## Memory/Disk Requirements

As the models are currently fully loaded into memory, you will need adequate disk space to save them and sufficient RAM to load them. At the moment, memory and disk requirements are the same.

| Model | Original size | Quantized size (Q4_0) |
|------:|--------------:|----------------------:|
|    7B |         13 GB |                3.9 GB |
|   13B |         24 GB |                7.8 GB |
|   30B |         60 GB |               19.5 GB |
|   65B |        120 GB |               38.5 GB |

## Quantization

Several quantization methods are supported. They differ in the resulting model disk size and inference speed.

The quantization formats `Q4_0_4_4`, `Q4_0_4_8` and `Q4_0_8_8` are block interleaved variants of the `Q4_0` format, providing a data layout that is better suited for specific implementations of optimized mulmat kernels. Since these formats differ only in data layout, they have the same quantized size as the `Q4_0` format.

*(outdated)*

| Model | Measure      |    F16 |   Q4_0 |   Q4_1 |   Q5_0 |   Q5_1 |   Q8_0 |
|------:|--------------|-------:|-------:|-------:|-------:|-------:|-------:|
|    7B | perplexity   | 5.9066 | 6.1565 | 6.0912 | 5.9862 | 5.9481 | 5.9070 |
|    7B | file size    |  13.0G |   3.5G |   3.9G |   4.3G |   4.7G |   6.7G |
|    7B | ms/tok @ 4th |    127 |     55 |     54 |     76 |     83 |     72 |
|    7B | ms/tok @ 8th |    122 |     43 |     45 |     52 |     56 |     67 |
|    7B | bits/weight  |   16.0 |    4.5 |    5.0 |    5.5 |    6.0 |    8.5 |
|   13B | perplexity   | 5.2543 | 5.3860 | 5.3608 | 5.2856 | 5.2706 | 5.2548 |
|   13B | file size    |  25.0G |   6.8G |   7.6G |   8.3G |   9.1G |    13G |
|   13B | ms/tok @ 4th |      - |    103 |    105 |    148 |    160 |    131 |
|   13B | ms/tok @ 8th |      - |     73 |     82 |     98 |    105 |    128 |
|   13B | bits/weight  |   16.0 |    4.5 |    5.0 |    5.5 |    6.0 |    8.5 |

- [k-quants](https://github.com/ggerganov/llama.cpp/pull/1684)
- recent k-quants improvements and new i-quants
  - [#2707](https://github.com/ggerganov/llama.cpp/pull/2707)
  - [#2807](https://github.com/ggerganov/llama.cpp/pull/2807)
  - [#4773 - 2-bit i-quants (inference)](https://github.com/ggerganov/llama.cpp/pull/4773)
  - [#4856 - 2-bit i-quants (inference)](https://github.com/ggerganov/llama.cpp/pull/4856)
  - [#4861 - importance matrix](https://github.com/ggerganov/llama.cpp/pull/4861)
  - [#4872 - MoE models](https://github.com/ggerganov/llama.cpp/pull/4872)
  - [#4897 - 2-bit quantization](https://github.com/ggerganov/llama.cpp/pull/4897)
  - [#4930 - imatrix for all k-quants](https://github.com/ggerganov/llama.cpp/pull/4930)
  - [#4951 - imatrix on the GPU](https://github.com/ggerganov/llama.cpp/pull/4957)
  - [#4969 - imatrix for legacy quants](https://github.com/ggerganov/llama.cpp/pull/4969)
  - [#4996 - k-qunats tuning](https://github.com/ggerganov/llama.cpp/pull/4996)
  - [#5060 - Q3_K_XS](https://github.com/ggerganov/llama.cpp/pull/5060)
  - [#5196 - 3-bit i-quants](https://github.com/ggerganov/llama.cpp/pull/5196)
  - [quantization tuning](https://github.com/ggerganov/llama.cpp/pull/5320), [another one](https://github.com/ggerganov/llama.cpp/pull/5334), and [another one](https://github.com/ggerganov/llama.cpp/pull/5361)

**Llama 2 7B**

| Quantization | Bits per Weight (BPW) |
|--------------|-----------------------|
| Q2_K         | 3.35                  |
| Q3_K_S       | 3.50                  |
| Q3_K_M       | 3.91                  |
| Q3_K_L       | 4.27                  |
| Q4_K_S       | 4.58                  |
| Q4_K_M       | 4.84                  |
| Q5_K_S       | 5.52                  |
| Q5_K_M       | 5.68                  |
| Q6_K         | 6.56                  |

**Llama 2 13B**

Quantization | Bits per Weight (BPW)
-- | --
Q2_K | 3.34
Q3_K_S | 3.48
Q3_K_M | 3.89
Q3_K_L | 4.26
Q4_K_S | 4.56
Q4_K_M | 4.83
Q5_K_S | 5.51
Q5_K_M | 5.67
Q6_K | 6.56

**Llama 2 70B**

Quantization | Bits per Weight (BPW)
-- | --
Q2_K | 3.40
Q3_K_S | 3.47
Q3_K_M | 3.85
Q3_K_L | 4.19
Q4_K_S | 4.53
Q4_K_M | 4.80
Q5_K_S | 5.50
Q5_K_M | 5.65
Q6_K | 6.56

You can also use the [GGUF-my-repo](https://huggingface.co/spaces/ggml-org/gguf-my-repo) space on Hugging Face to build your own quants without any setup.
